In [83]:
from keras.models import model_from_json
from keras.layers import Conv3D, MaxPooling3D, UpSampling3D, BatchNormalization, Activation, Input
from keras.models import Model
from keras import backend as K
import numpy as np
from sklearn import preprocessing
import random
from random import randint
# random.seed(9001)

In [84]:
json_file = open('keras/model_2/model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()

In [85]:
loaded_model = model_from_json(loaded_model_json)
loaded_model.load_weights("keras/model_2/model.h5")
loaded_model.compile(optimizer='adadelta', loss='binary_crossentropy')

In [86]:
def train_test_split(split_part):
    og_data = np.load('images.npy')
    labels = np.load('labels.npy')
    
    # Label encoder
    le = preprocessing.LabelEncoder()
    le.fit(labels)
    labels = le.transform(labels)
    x_train, x_test, y_train, y_test = [], [], [], []

    norm_data = og_data/255
    # Sorting the data
    sorted_index = np.argsort(labels)
    sorted_labels = labels[sorted_index]
    sorted_og_data = og_data[sorted_index]
    sorted_norm_data = norm_data[sorted_index]

    # split the data into train and test
    test_random_indices = []
    for i in range(int(sorted_labels.shape[0]/split_part)):
        index = i*split_part + randint(0, split_part - 1)
        test_random_indices.append(index)

    x_train = np.delete(sorted_norm_data, test_random_indices, 0)
    x_test = sorted_norm_data[test_random_indices]
    y_train = np.delete(sorted_labels, test_random_indices)
    y_test = sorted_labels[test_random_indices]
    
    return x_train, x_test, y_train, y_test

In [87]:
print('Train test split')
x_train, x_test, y_train, y_test = train_test_split(6)

print('Preprocessing')
X = []
for i in range(x_train.shape[0]):
    X.append(np.pad(x_train[i], ((3, 2), (2, 1), (1, 0)), 'constant', constant_values=(0.0)))

X_t = []

for i in range(x_test.shape[0]):
    X_t.append(np.pad(x_test[i], ((3, 2), (2, 1), (1, 0)), 'constant', constant_values=(0.0))) 

X = np.array(X, dtype=float)
X = X.reshape(300, 56, 64, 24, 1)

X_t = np.array(X_t, dtype=float)
X_t = X_t.reshape(60, 56, 64, 24, 1)

Train test split
Preprocessing


In [20]:
# score = loaded_model.evaluate(X, X, verbose=1)

300/300 [==============================] - 134s 446ms/step


In [88]:
loaded_model.evaluate(X_t, X_t, verbose=1)

60/60 [==============================] - 25s 413ms/step


0.18124904135862988

In [89]:
input_shape = (56, 64, 24, 1)
input_img = Input(shape=input_shape)

# x = Conv3D(16, (3, 3, 3), padding='same')(input_img)
# x = BatchNormalization()(x)
# x = Activation(activation='relu')(x)
# x = MaxPooling3D((2, 2, 2), padding='same')(x)
# x = Conv3D(8, (3, 3, 3), padding='same')(x)
# x = BatchNormalization()(x)
# x = Activation(activation='relu')(x)
# x = MaxPooling3D((2, 2, 2), padding='same')(x)
# x = Conv3D(8, (3, 3, 3), padding='same')(x)
# x = BatchNormalization()(x)
# x = Activation(activation='relu')(x)


x = Conv3D(16, (3, 3, 3), activation='relu', padding='same')(input_img)
x = MaxPooling3D((2, 2, 2), padding='same')(x)
x = Conv3D(8, (3, 3, 3), activation='relu', padding='same')(x)
x = MaxPooling3D((2, 2, 2), padding='same')(x)
x = Conv3D(8, (3, 3, 3), activation='relu', padding='same')(x)

encoded = MaxPooling3D((2, 2, 2), padding='same')(x)

In [91]:
encoder = Model(input_img, encoded)

Xtr = encoder.predict(X)
Xts = encoder.predict(X_t)

In [42]:
fmri_batch = dict()

fmri_batch['Xtr'] = Xtr
fmri_batch['Xts'] = Xts
fmri_batch['Ytr'] = y_train
fmri_batch['Yts'] = y_test

In [44]:
np.save('keras/model_1/data', fmri_batch)

In [45]:
data = np.load('keras/model_1/data.npy')

In [92]:
Xtr = Xtr.reshape(300, 1344)
Xts = Xts.reshape(60, 1344)

In [93]:
from sklearn.svm import LinearSVC

In [95]:
clf = LinearSVC()
clf.fit(Xtr, y_train)

results_top = []
for i in [1, 3, 5, 10]:
	predicted_correct = 0
	for k in range(y_test.shape[0]):
		o = Xts[k]
		o = o.reshape(1, -1)
		decision_lbl = clf.decision_function(o)[0]
		top_class = i
		top_n = np.argsort(decision_lbl)[::-1][:top_class]
		if y_test[k] in top_n:
			predicted_correct += 1.0/60
	results_top.append(predicted_correct)

print(results_top)

[0.11666666666666665, 0.24999999999999997, 0.31666666666666665, 0.44999999999999996]
